In [1]:
import pandas as pd 
import torch
import numpy as np

dfRaw = pd.read_csv("macro40.csv") 

In [2]:
### "Data preprocessing
dfP = dfRaw.drop("Date",axis=1) # only in the first time
df = dfP.sub(dfP.mean(axis=0), axis=1).div(dfP.std(0),axis=1)

In [3]:
r1 = 4
r2 = 3
r3 = 2
p = 4
N = 40
Smp_size = 100
Pred_size = len(df.index)-Smp_size-p

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from scipy.linalg import svd
from torch.utils.data import Dataset, DataLoader
from numpy import linalg as LA
from time import perf_counter 
from torch.autograd import Variable
import torch.optim as optim

### '' Define the Network Structure: Linear
### Type 1. Linear Network
class NetLinear(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetLinear, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=False) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=False)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=False)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=False)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        z = self.conv2(x[:, :1, :, :])
        for i in range(1, x.shape[1]):
            z = torch.cat([z, self.conv2(x[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z = z.view(-1, self.r2*self.r3) #-1 helps us figure out the batchsize
        x = self.fc1(z) #activation can be added on the inside as well
        x = self.fc2(x)
        return x
    
### Type 2. Nonlinear Network
class NetRelu(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetRelu, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=True) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=True)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=True)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=True)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        z = F.relu(self.conv2(x[:, :1, :, :]))
        for i in range(1, x.shape[1]):
            z = torch.cat([z, self.conv2(x[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z = z.view(-1, self.r2*self.r3) #-1 helps us figure out the batchsize
        x = self.fc1(z) #activation can be added on the inside as well
        x = self.fc2(x)
        return x
    
### Type 3. Nonlinear DW Network    
class NetDW(nn.Module):

    def __init__(self, r1, r2, r3, p, N):
        
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.p = p
        self.N = N
        super(NetDW, self).__init__()
        # .conv1: 1 input matrix channel (N*P), r2 output channels, Nx1 convolution kernel
        # .conv2: 1 input matrix channel (1*P), r3 output channels, 1xr3 convolution kernel (kernel sharing)
        self.conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=True) # stride is set to be (0,1) -> only move to the right
        self.conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=True)   # stride is set to be 0 -> no moving needed
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=True)  # 6*6 from image dimension
        self.fc2 = nn.Linear(in_features=r1, out_features=N, bias=True)

    def forward(self, x):
        y1 = F.relu(self.conv1(x))
        y2 = F.relu(self.conv2(x))
        # first N, then p
        z1 = self.conv2(y1[:, :1, :, :])
        for i in range(1, y1.shape[1]):
            z1 = torch.cat([z1, self.conv2(y1[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
        z1 = F.relu(z1.view(-1, self.r2*self.r3)) #-1 helps us figure out the batchsize
        # first p, then N
        z2 = F.relu(self.conv1(y2[:, :1, :, :]))
        for i in range(1, y2.shape[1]):
            z2 = torch.cat([z2, self.conv1(y2[:, i:(i+1), :, :])], dim = 2) #Flattening is achieved     
        z2 = F.relu(z2.view(-1, self.r2*self.r3)) #-1 helps us figure out the batchsize
        x1 = self.fc2(F.relu(self.fc1(z1)))
        x2 = self.fc2(F.relu(self.fc1(z2)))
        x = torch.stack([x1,x2])
        x = torch.mean(x,dim=0)
        return x

# Type 4: MLP
class MLP(nn.Module):
    
    def __init__(self, p, N):
        self.p = p
        self.N = N
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features = N*p, out_features = N, bias = True)
    def forward(self, x):
        y = self.fc1(x)
        return y
    
# Type 5: RRR 
class RRR(nn.Module):
    
    def __init__(self, p, N, r1):
        self.p = p
        self.N = N
        self.r1 = r1
        super(RRR, self).__init__()
        self.fc1 = nn.Linear(in_features = N*p, out_features = r1, bias = True)
        self.fc2 = nn.Linear(in_features = r1, out_features = N, bias = True)
    def forward(self, x):
        x = self.fc1(x)
        y = self.fc2(x)
        return y

In [16]:
def kronecker(A, B):
    return torch.ger(A.view(-1), B.view(-1)).reshape(*(A.size() + B.size())).permute([0, 2, 1, 3]).reshape(A.size(0)*B.size(0),A.size(1)*B.size(1))

### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(torch.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = max(abs(torch.squeeze(A_Est-A_True))).item()
        return gap
    
def rearrangeG(K):
    K = K.permute(1,0)
    i = j = 0
    for j in range(r3):
            for i in range(r2):
                if i == 0 and j == 0:
                    tmp = K[:1,:]
                else:
                    tmp = torch.cat([tmp, K[(r3*i+j):(r3*i+1+j),:]], dim = 0)   
    tmp = tmp.permute(1,0)
    return(tmp)

def Param_Matrix(net):
    
    U2T = torch.squeeze(net.conv1.weight)
    U3T = torch.squeeze(net.conv2.weight)
    G1 = rearrangeG(net.fc1.weight)
    U1 = net.fc2.weight
    A = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
    
    return A

def Param_Dict(net):
    
    U2T = torch.squeeze(net.conv1.weight)
    U3T = torch.squeeze(net.conv2.weight)
    G1 = rearrangeG(net.fc1.weight)
    U1 = net.fc2.weight
    A = torch.mm(torch.mm(U1, G1), kronecker(U3T,U2T))
    
    Param_List = {
        "U2T" : U2T,
        "U3T" : U3T,
        "G1" : G1,
        "U1" : U1,
        "A" : A
    }
    
    return Param_List

    #A = genA_true.genA(r1, r2, r3, p, N) #generate large transition matrix 

In [17]:
### "We can use our method to generate RandomDataset
# For our linear settings burnt in is needed
class RealDataset(Dataset):
    
    def __init__(self, p, N, Smp_size, df):
        self.X = []
        self.y = []
        for i in range(Smp_size):
            if i == 0:
                input_TS = torch.tensor(df.iloc[ [3,2,1,0] , : ].values).permute(1,0).view(1,1,N,p)
                self.X.append(torch.squeeze(input_TS.view(1,1,N,p), dim = 0))
                output_TS = torch.squeeze(torch.tensor(df.iloc[ p:(p+1) , : ].values).permute(1,0).view(1,1,N,1))
                self.y.append(output_TS[:N])  
            else:
                input_TS = torch.cat([self.y[i-1].view(1,1,N,1), input_TS], dim = 3)
                self.X.append(torch.squeeze(input_TS[:,:,:N,:p], dim = 0))
                out_tmp = torch.squeeze(torch.tensor(df.iloc[ (i+p):(i+p+1) , : ].values).permute(1,0).view(1,1,N,1))
                self.y.append(out_tmp[:N])
                
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

In [18]:
# input to GPU
#device = torch.device("cuda:2")
#device2 = torch.device('cpu')

In [25]:
# initialize dict with dynamic list for storage
names = {}

names["y_true"] = []

names["predErrorDW"] =  []
names["predErrorDWLinf"] =  []
names["LTRDW_pred"] = []
names["predErrorN"] =  []
names["predErrorNLinf"] =  []
names["LTRN_pred"] = []
names["predErrorL"] =  []
names["predErrorLLinf"] =  []
names["LTRL_pred"] = []
names["predErrorM"] =  []
names["predErrorMLinf"] =  []
names["LTRM_pred"] = []
names["predErrorR"] =  []
names["predErrorRLinf"] =  []
names["LTRR_pred"] = []

In [ ]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(Pred_size):
    
    if k == 0:
        dsT = RealDataset(p=p, N=N, Smp_size=Smp_size+k, df=df)
    else:
        dsT = dsF
        
    dsF = RealDataset(p=p, N=N, Smp_size=Smp_size+k+1, df=df)
    X_F,y_F = dsF[Smp_size+k]
    X_F = X_F.view(1,1,N,p)
    names["y_true"].append(y_F)
    ds = DataLoader(dsT, batch_size=Smp_size+k, shuffle=False)

    netDW = NetDW(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerDW = optim.SGD(netDW.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    ### "" netDW
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netDW(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerDW.zero_grad() 
            loss.backward() 
            optimizerDW.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predDW = netDW(X_F).float()
    predErrorDW = distance(y_predDW, y_F)
    names["predErrorDW"].append(predErrorDW)
    print("PredError is {}.".format(predErrorDW))
    predErrorDWLinf = distanceLinf(y_predDW, y_F)
    names["predErrorDWLinf"].append(predErrorDWLinf)
    print("PredErrorLinf is {}.".format(predErrorDWLinf))
    names["LTRDW_pred"].append(y_predDW)
    
    ### "" netReLU
    netN = NetRelu(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerN = optim.SGD(netN.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netN(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerN.zero_grad() 
            loss.backward() 
            optimizerN.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predN = netN(X_F).float()
    predErrorN = distance(y_predN, y_F)
    names["predErrorN"].append(predErrorN)
    print("PredError Nonlinear is {}.".format(predErrorN))
    predErrorNLinf = distanceLinf(y_predN, y_F)
    names["predErrorNLinf"].append(predErrorNLinf)
    print("PredErrorLinf Nonlinear is {}.".format(predErrorNLinf))
    names["LTRN_pred"].append(y_predN)
    
    ### "" netLinear
    netL = NetLinear(r1, r2, r3, p, N)

    criterion = nn.MSELoss()
    optimizerL = optim.SGD(netL.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netL(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerL.zero_grad() 
            loss.backward() 
            optimizerL.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predL = netL(X_F).float()
    predErrorL = distance(y_predL, y_F)
    names["predErrorL"].append(predErrorL)
    print("PredError Linear is {}.".format(predErrorL))
    predErrorLLinf = distanceLinf(y_predN, y_F)
    names["predErrorLLinf"].append(predErrorLLinf)
    print("PredErrorLinf Linear is {}.".format(predErrorLLinf))
    names["LTRL_pred"].append(y_predL)
    
    ### "" MLP
    netM = MLP(p, N)

    criterion = nn.MSELoss()
    optimizerM = optim.SGD(netM.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = torch.squeeze(_x.permute(0,1,3,2).reshape(Smp_size+k, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netM(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerM.zero_grad() 
            loss.backward() 
            optimizerM.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F.permute(0,1,3,2).reshape(1, p*N)).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predM = netM(X_F).float()
    predErrorM = distance(y_predM, y_F)
    names["predErrorM"].append(predErrorM)
    print("PredError MLP is {}.".format(predErrorM))
    predErrorMLinf = distanceLinf(y_predM, y_F)
    names["predErrorMLinf"].append(predErrorMLinf)
    print("PredErrorLinf MLP is {}.".format(predErrorMLinf))
    names["LTRM_pred"].append(y_predM)

    ### "" RRR
    netR = RRR(p, N, r1)

    criterion = nn.MSELoss()
    optimizerR = optim.SGD(netR.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = torch.squeeze(_x.permute(0,1,3,2).reshape(Smp_size+k, 1, p*N).permute(1, 0, 2))
            _x = Variable(_x).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = netR(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizerR.zero_grad() 
            loss.backward() 
            optimizerR.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_predR = netR(X_F).float()
    predErrorR = distance(y_predR, y_F)
    names["predErrorR"].append(predErrorR)
    print("PredError RRR is {}.".format(predErrorR))
    predErrorRLinf = distanceLinf(y_predR, y_F)
    names["predErrorRLinf"].append(predErrorRLinf)
    print("PredErrorLinf RRR is {}.".format(predErrorRLinf))
    names["LTRR_pred"].append(y_predR)

    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100s = names
torch.save(Real_100s, "Real_100s.py")

/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError is 5.987775110602141.
PredErrorLinf is 2.0030364990234375.


/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError Nonlinear is 5.849097105595324.
PredErrorLinf Nonlinear is 2.332639217376709.


/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

PredError MLP is 11.005014490363267.
PredErrorLinf MLP is 4.380699157714844.


/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError RRR is 6.745110612984526.
PredErrorLinf RRR is 2.2803232669830322.
0
PredError is 4.925665857837537.
PredErrorLinf is 2.452010154724121.
PredError Nonlinear is 4.98278591930038.
PredErrorLinf Nonlinear is 2.452010154724121.
PredError Linear is 8.33945128614025.
PredErrorLinf Linear is 2.452010154724121.
PredError MLP is 8.868925782948972.
PredErrorLinf MLP is 3.712214469909668.
PredError RRR is 6.220670010704641.
PredErrorLinf RRR is 2.840993881225586.
1
PredError is 5.7782162516244675.
PredErrorLinf is 2.6528213024139404.
PredError Nonlinear is 11.61960140102706.
PredErrorLinf Nonlinear is 3.6993327140808105.
PredError Linear is 8.59238048391545.
PredErrorLinf Linear is 3.6993327140808105.
PredError MLP is 11.60756236900714.
PredErrorLinf MLP is 3.7524566650390625.
PredError RRR is 10.292894256727589.
PredErrorLinf RRR is 2.7696127891540527.
2
PredError is 6.028791331656313.
PredErrorLinf is 2.2714319229125977.
PredError Nonlinear is 6.719338750548964.
PredErrorLinf Nonlinea

PredError MLP is 7.911021846929615.
PredErrorLinf MLP is 3.636488199234009.
PredError RRR is 6.005239106689727.
PredErrorLinf RRR is 3.6907756328582764.
21
PredError is 9.204193105065324.
PredErrorLinf is 2.953857660293579.
PredError Nonlinear is 9.417581519413819.
PredErrorLinf Nonlinear is 2.953857660293579.
PredError Linear is 7.116945499195943.
PredErrorLinf Linear is 2.953857660293579.
PredError MLP is 11.736272706694338.
PredErrorLinf MLP is 5.143171787261963.
PredError RRR is 9.42390726164959.
PredErrorLinf RRR is 3.277275800704956.
22
PredError is 5.590144695122152.
PredErrorLinf is 2.4634668827056885.
PredError Nonlinear is 5.574088209370634.
PredErrorLinf Nonlinear is 2.4634668827056885.
PredError Linear is 4.510852231791201.
PredErrorLinf Linear is 2.4634668827056885.
PredError MLP is 11.919923635148567.
PredErrorLinf MLP is 5.270861625671387.
PredError RRR is 8.037346926547078.
PredErrorLinf RRR is 2.9169788360595703.
23
PredError is 4.067277886364199.
PredErrorLinf is 1.81

PredError Linear is 4.938847804509509.
PredErrorLinf Linear is 1.6733179092407227.
PredError MLP is 9.61086632546093.
PredErrorLinf MLP is 3.5975453853607178.
PredError RRR is 5.801715649700567.
PredErrorLinf RRR is 1.9741986989974976.
42
PredError is 3.825268734735422.
PredErrorLinf is 1.6548806428909302.
PredError Nonlinear is 3.9027749435275743.
PredErrorLinf Nonlinear is 1.6548806428909302.
PredError Linear is 4.785055603214501.
PredErrorLinf Linear is 1.6548806428909302.
PredError MLP is 9.916844812497382.
PredErrorLinf MLP is 3.6669504642486572.
PredError RRR is 6.85927517412056.
PredErrorLinf RRR is 1.9603060483932495.
43
PredError is 3.917845704113764.
PredErrorLinf is 1.7315759658813477.
PredError Nonlinear is 3.4634481240549886.
PredErrorLinf Nonlinear is 1.2524359226226807.
PredError Linear is 4.034999788986258.
PredErrorLinf Linear is 1.2524359226226807.
PredError MLP is 8.28034294487058.
PredErrorLinf MLP is 3.2859907150268555.
PredError RRR is 4.940856179240147.
PredError

In [126]:
from statistics import mean,median

print(mean(names["predErrorDW"]))
print(mean(names["predErrorDWLinf"]))
print(mean(names["predErrorN"]))
print(mean(names["predErrorNLinf"]))
print(mean(names["predErrorL"]))
print(mean(names["predErrorLLinf"]))
print(mean(names["predErrorM"]))
print(mean(names["predErrorMLinf"]))
print(mean(names["predErrorR"]))
print(mean(names["predErrorRLinf"]))

5.485155231298449
2.4370612335205077


In [104]:
conv1 = nn.Conv2d(1, r2, kernel_size=(N, 1), bias=False) # stride is set to be (0,1) -> only move to the right
conv2 = nn.Conv2d(1, r3, kernel_size=(1, p), bias=False)   # stride is set to be 0 -> no moving needed

fc1 = nn.Linear(in_features=r2*r3, out_features=r1, bias=False)  # 6*6 from image dimension
fc2 = nn.Linear(in_features=r1, out_features=N, bias=False)
y1 = F.relu(conv1(x))
y2 = F.relu(conv2(x))

z1 = conv2(y1[:, :1, :, :])
for i in range(1, y1.shape[1]):
    z1 = torch.cat([z1, conv2(y1[:, i:(i+1), :, :])], dim = 1) #Flattening is achieved     
z1 = F.relu(z1.view(-1, r2*r3)) #-1 helps us figure out the batchsize

z2 = F.relu(conv1(y2[:, :1, :, :]))
for i in range(1, y2.shape[1]):
    z2 = torch.cat([z2, conv1(y2[:, i:(i+1), :, :])], dim = 2) #Flattening is achieved     
z2 = F.relu(z2.view(-1, r2*r3)) #-1 helps us figure out the batchsize

x1 = F.relu(fc2(F.relu(fc1(z1))))
x2 = F.relu(fc2(F.relu(fc1(z2))))
x = torch.stack([x1,x2])
print(x.shape)
x = torch.mean(x,dim=0)
print(x.shape)

torch.Size([2, 165, 40])
torch.Size([165, 40])
